In [ ]:
import pandas as pd
import datetime as dt
import time
import calendar

from uce_resources import get_site_id, get_forecast, put_forecast

In [ ]:
from settings.sites import ceg as sites_list

target_year = 2023
target_month = 1
forecasts_types = ['real', 'naive', 'zero']

# sites_list = ['Balivka']
sites_data = dict.fromkeys(sites_list)
print(sites_data)

In [ ]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from settings.db import DO_URL

engine = create_engine(DO_URL, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

In [ ]:
first_date = dt.date(target_year, target_month, 1)
last_date = dt.date(target_year, target_month, calendar.monthrange(target_year, target_month)[1])
# first_date = dt.date(target_year, target_month, 1)
# last_date = dt.date(target_year, target_month, 4)

target_dates = pd.date_range(start=first_date, end=last_date, freq='D').to_pydatetime()

In [ ]:

with engine.connect() as connection:
    for date in target_dates:    
        for site in sites_data.keys():
            start = time.time()
            print('-'*50)
            print(site)
            site_id, legal_entity = get_site_id(site, connection, 
                                                metadata.tables['sites'],
                                                include_legal_entity_id=True)
            restored_forecast = get_forecast(site_id, [date], 'restored', connection, metadata)
            update_time = dt.datetime.utcnow()

            time_indexes = restored_forecast.index.to_pydatetime()
            data_values = [int(value * 1000) for value in restored_forecast.tolist()]
            hours = range(1, len(data_values) + 1)

            put_forecast(
                site_id,
                "forecast_applied_raw",
                date.date(),
                hours,
                time_indexes,
                data_values,
                update_time,
                connection=connection
            )

            print(f'Raw forecast is writen: {site} | {date}')

